In [18]:
import sys
sys.path.append("..")

In [19]:
import re
from collections import Counter

from resources.breadth_first_solver.breadth_first_solver import BreadthFirstSolver
from resources.breadth_first_solver.game import Game
from resources.utils import get_puzzle_input

### Part 1


Grab operations from Day 14

In [20]:
ops = []

### Addition

```
addr (add register) stores into register C the result of adding register A and register B.
addi (add immediate) stores into register C the result of adding register A and value B.
```

In [21]:
def addr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] + registers[B]
    return tuple(output)

def addi(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] + B
    return tuple(output)

ops.extend((addr, addi))

### Multiplication

```
mulr (multiply register) stores into register C the result of multiplying register A and register B.
muli (multiply immediate) stores into register C the result of multiplying register A and value B.
```

In [22]:
def mulr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] * registers[B]
    return tuple(output)

def muli(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] * B
    return tuple(output)

ops.extend((mulr, muli))

### Bitwise AND

```
banr (bitwise AND register) stores into register C the result of the bitwise AND of register A and register B.
bani (bitwise AND immediate) stores into register C the result of the bitwise AND of register A and value B.
```

In [23]:
def banr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] & registers[B]
    return tuple(output)

def bani(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] & B
    return tuple(output)

ops.extend((banr, bani))

### Bitwise OR

```
borr (bitwise OR register) stores into register C the result of the bitwise OR of register A and register B.
bori (bitwise OR immediate) stores into register C the result of the bitwise OR of register A and value B.
```

In [24]:
def borr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] | registers[B]
    return tuple(output)

def bori(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] | B
    return tuple(output)

ops.extend((borr, bori))

### Assignment

```
setr (set register) copies the contents of register A into register C. (Input B is ignored.)
seti (set immediate) stores value A into register C. (Input B is ignored.)
```


In [25]:
def setr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions
    output[C] = registers[A]
    return tuple(output)

def seti(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions
    output[C] = A
    return tuple(output)

ops.extend((setr, seti))

### Greater-than testing

```
gtir (greater-than immediate/register) sets register C to 1 if value A is greater than register B. Otherwise, register C is set to 0.
gtri (greater-than register/immediate) sets register C to 1 if register A is greater than value B. Otherwise, register C is set to 0.
gtrr (greater-than register/register) sets register C to 1 if register A is greater than register B. Otherwise, register C is set to 0.
```

In [26]:
def gtir(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(A > registers[B])
    return tuple(output)

def gtri(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] > B)
    return tuple(output)

def gtrr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] > registers[B])
    return tuple(output)

ops.extend((gtir, gtri, gtrr))

### Equality testing

```
eqir (equal immediate/register) sets register C to 1 if value A is equal to register B. Otherwise, register C is set to 0.
eqri (equal register/immediate) sets register C to 1 if register A is equal to value B. Otherwise, register C is set to 0.
eqrr (equal register/register) sets register C to 1 if register A is equal to register B. Otherwise, register C is set to 0.
```

In [27]:
def eqir(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(A == registers[B])
    return tuple(output)

def eqri(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] == B)
    return tuple(output)

def eqrr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] == registers[B])
    return tuple(output)

ops.extend((eqir, eqri, eqrr))

In [28]:
[op.__name__ for op in ops]

['addr',
 'addi',
 'mulr',
 'muli',
 'banr',
 'bani',
 'borr',
 'bori',
 'setr',
 'seti',
 'gtir',
 'gtri',
 'gtrr',
 'eqir',
 'eqri',
 'eqrr']

In [29]:
def parse_input(lines):
    ip = 0
    instructions = []
    for line in lines:
        parts = line.split(' ')        
        if len(parts) == 2:
            ip = int(parts[1])
        else:
            instructions.append((parts[0],) + tuple(int(i) for i in parts[1:]))
            
    return ip, instructions

In [38]:
class InstructionRunner:

    def __init__(self, instructions, instruction_register, part_1=True):
        self.instructions = instructions
        self.instruction_register = instruction_register

        if part_1:
            self.registers = [0, 0, 0, 0, 0, 0]
        else:
            self.registers = [1, 0, 0, 0, 0, 0]

        self.pointer = 0
        self.op_map = {op.__name__: op for op in ops}
        self.num_moves = 0
        self.finished = False
        self.seen_instructions = {}
        self.instruction_counter = Counter()
                
    def move(self):
        if self.finished:
            raise ValueError('finished...')

        # print('registers: {}'.format(self.registers))
        try:
            instruction = self._instruction  #1
            self.update_instruction_register()  #2
            
            op = self.op_map[instruction[0]]
            self.registers = list(op(self.registers, instruction))  #3

        except IndexError:
            self.finished = True

        self.update_instruction_pointer() # 4
        
        if self.num_moves % 1000000 == 0:
            print('Move {}, Registers: {}, Executed: {}'.format(
                self.num_moves,
                self.registers,
                self.instruction_counter
            ))
            print()

        self.num_moves += 1

    def update_instruction_register(self):
        self.registers[self.instruction_register] = self.pointer
        
    def update_instruction_pointer(self):
        self.pointer = self.registers[self.instruction_register]
        self.pointer += 1
    
    @property
    def _instruction(self):
        instruction = self.instructions[self.pointer]
        self.instruction_counter[self.pointer] += 1
        # print('Running', self.pointer, instruction)
        return instruction        

In [39]:
test_input="""#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5""".split('\n')

In [40]:
test_register, test_instructions = parse_input(test_input)
test_register, test_instructions

(0,
 [('seti', 5, 0, 1),
  ('seti', 6, 0, 2),
  ('addi', 0, 1, 0),
  ('addr', 1, 2, 3),
  ('setr', 1, 0, 0),
  ('seti', 8, 0, 4),
  ('seti', 9, 0, 5)])

In [41]:
test_runner = InstructionRunner(test_instructions, test_register)

In [42]:
test_runner = InstructionRunner(test_instructions, test_register)
while not test_runner.finished:
    test_runner.move()
    print(test_runner.registers)
    

Move 0, Registers: [0, 5, 0, 0, 0, 0], Executed: Counter({0: 1})

[0, 5, 0, 0, 0, 0]
[1, 5, 6, 0, 0, 0]
[3, 5, 6, 0, 0, 0]
[5, 5, 6, 0, 0, 0]
[6, 5, 6, 0, 0, 9]
[6, 5, 6, 0, 0, 9]


In [43]:
puzzle_input = get_puzzle_input('/tmp/day_19.txt')
puzzle_register, puzzle_instructions = parse_input(puzzle_input)

In [45]:
puzzle_runner = InstructionRunner(
    puzzle_instructions,
    puzzle_register,
    part_1=True
)
while not puzzle_runner.finished:
    puzzle_runner.move()
# for _ in range(100):
#     puzzle_runner.move()

print(puzzle_runner.registers)

Move 0, Registers: [0, 0, 16, 0, 0, 0], Executed: Counter({0: 1})

Move 1000000, Registers: [114, 909, 2, 398, 0, 138], Executed: Counter({3: 124930, 4: 124930, 5: 124930, 8: 124930, 9: 124930, 10: 124930, 6: 124926, 11: 124793, 2: 138, 12: 137, 13: 137, 14: 137, 15: 137, 7: 4, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 1: 1})

Move 2000000, Registers: [114, 909, 7, 796, 0, 275], Executed: Counter({3: 249862, 4: 249862, 5: 249862, 8: 249861, 9: 249861, 10: 249861, 6: 249858, 11: 249587, 2: 275, 12: 274, 13: 274, 14: 274, 15: 274, 7: 4, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 1: 1})

Move 3000000, Registers: [417, 909, 7, 285, 0, 413], Executed: Counter({3: 374793, 4: 374793, 5: 374793, 8: 374792, 9: 374792, 10: 374792, 6: 374788, 11: 374380, 2: 413, 12: 412, 13: 412, 14: 412, 15: 412, 7: 5, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 1: 1})

Move 4000000, Registers: [417, 909, 2, 684, 0

### Part 2

Now with a 1 in buffer 0

In [65]:
puzzle_runner = InstructionRunner(puzzle_instructions, puzzle_register, part_1=False)
while not puzzle_runner.finished:
    puzzle_runner.move()

print(puzzle_runner.registers)

Move 0, Registers: [1, 0, 16, 0, 0, 0], Executed: Counter({0: 1})

Move 1000000, Registers: [0, 10551309, 7, 124998, 0, 1], Executed: Counter({3: 124998, 4: 124998, 5: 124998, 6: 124998, 8: 124997, 9: 124997, 10: 124997, 11: 124997, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})

Move 2000000, Registers: [0, 10551309, 7, 249998, 0, 1], Executed: Counter({3: 249998, 4: 249998, 5: 249998, 6: 249998, 8: 249997, 9: 249997, 10: 249997, 11: 249997, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})

Move 3000000, Registers: [0, 10551309, 7, 374998, 0, 1], Executed: Counter({3: 374998, 4: 374998, 5: 374998, 6: 374998, 8: 374997, 9: 374997, 10: 374997, 11: 374997, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1}

Move 27000000, Registers: [0, 10551309, 7, 3374998, 0, 1], Executed: Counter({3: 3374998, 4: 3374998, 5: 3374998, 6: 3374998, 8: 3374997, 9: 3374997, 10: 3374997, 11: 3374997, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})

Move 28000000, Registers: [0, 10551309, 7, 3499998, 0, 1], Executed: Counter({3: 3499998, 4: 3499998, 5: 3499998, 6: 3499998, 8: 3499997, 9: 3499997, 10: 3499997, 11: 3499997, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})

Move 29000000, Registers: [0, 10551309, 7, 3624998, 0, 1], Executed: Counter({3: 3624998, 4: 3624998, 5: 3624998, 6: 3624998, 8: 3624997, 9: 3624997, 10: 3624997, 11: 3624997, 0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})

Move 30000000, Registers: [0, 1055

KeyboardInterrupt: 

Examining the registers after running for some time:

```
Move 39000000, Registers: [0, 10551309, 7, 4874998, 0, 1],
Executed: Counter(
    {3: 4874998,
     4: 4874998,
     5: 4874998,
     6: 4874998,
     8: 4874997,
     9: 4874997,
     10: 4874997,
     11: 4874997,
     0: 1,
     17: 1,
     18: 1,
     19: 1, 
     20: 1,
     21: 1,
     22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})

Move 40000000, Registers: [0, 10551309, 7, 4999998, 0, 1],
Executed: Counter(
    {3: 4999998,
     4: 4999998,
     5: 4999998,
     6: 4999998,
     8: 4999997,
     9: 4999997,
     10: 4999997,
     11: 4999997,
     0: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 1: 1, 2: 1})
```

Stuck in loop execution 3, 4, 5, 6, 7, 9, 10 and 11 with pointer at 7

```
3  mulr 5 3 4
4  eqrr 4 1 4
5  addr 4 2 2
6  addi 2 1 2  # Jump 
7  addr 5 0 0
8  addi 3 1 3
9  gtrr 3 1 4
10 addr 2 4 2
11 seti 2 8 2
```

```
pointer = 3
inst 
#3 reg_4 = reg_5 * reg_3
#4 


Puzzle input

```
#ip 2
0  addi 2 16 2 * -> 17 Block B

# Block D
1  seti 1 1 5    -> reg[5] = 1
2  seti 1 1 3    -> reg[3] = 1
3  mulr 5 3 4    -> reg[4] = reg[3] * reg[5]
4  eqrr 4 1 4    -> reg[4] = reg[4] == reg[1]
5  addr 4 2 2  *
6  addi 2 1 2  * -> 8
7  addr 5 0 0
8  addi 3 1 3
9  gtrr 3 1 4
10 addr 2 4 2  *
11 seti 2 8 2  * -> 3
12 addi 5 1 5
13 gtrr 5 1 4
14 addr 4 2 2  *
15 seti 1 5 2  * -> 2
16 mulr 2 2 2  * -> break!!!

# block_B
17 addi 1 2 1
18 mulr 1 1 1
19 mulr 2 1 1
20 muli 1 11 1

# Block_C
21 addi 4 3 4
22 mulr 4 2 4
23 addi 4 7 4
24 addr 1 4 1

25 addr 2 0 2  * idx += reg[0] -> 27 with part 2
26 seti 0 4 2  * -> 1

block_A
x = 27
27 setr 2 8 4    -> reg[4] = x
28 mulr 4 2 4    -> reg[4] = x(x+1)
29 addr 2 4 4    -> reg[4] = x + 2 + x(x+1) = x^2 + 2x + 2 
30 mulr 2 4 4    -> reg[4] = (x+3)(x^2 + 2x + 2) = x^3 +6x^2 + 6
31 muli 4 14 4   -> reg[4] = 14x^3 +84x^2 + 84
32 mulr 4 2 4    -> reg[4] = (x+5)(14x^3 +84x^2 + 84) = 14x^4 + 154x^3 + 420x^2 + 84x + 420
# sets reg[4] to 10550400

33 addr 1 4 1    -> reg[1] += 10550400

...
34 seti 0 5 0    -> reg[0] = 0
35 seti 0 8 2  * -> 1
```

In [1]:
# 27 setr 2 8 4    -> reg[4] = x
# 28 mulr 4 2 4    -> reg[4] = x(x+1)
# 29 addr 2 4 4    -> reg[4] = x + 2 + x(x+1) = x^2 + 2x + 2 
# 30 mulr 2 4 4    -> reg[4] = (x+3)(x^2 + 2x + 2) = x^3 +6x^2 + 6
# 31 muli 4 14 4   -> reg[4] = 14x^3 +84x^2 + 84
# 32 mulr 4 2 4    -> reg[4] = (x+5)(14x^3 +84x^2 + 84) = 14x^4 + 154x^3 + 420x^2 + 84x + 420

# sets reg[4] to 10550400
def block_A():
    x = 27
    res = x
    res *= (x + 1)
    res += (x + 2)
    res *= (x + 3)
    res *= 14
    res *= (x + 5)
    return res

In [2]:
block_A()

10550400

In [3]:
# block_B
# 17 addi 1 2 1
# 18 mulr 1 1 1
# 19 mulr 2 1 1
# 20 muli 1 11 1
def block_B(buf_1):
    x = 17
    buf_1 += 2
    buf_1 *= buf_1
    buf_1 *= (x + 2)
    buf_1 *= 11 
    return buf_1

In [4]:
# registers: [1, 0, 16, 0, 0, 0]
# Running 17 ('addi', 1, 2, 1)
# registers: [1, 2, 17, 0, 0, 0]
# Running 18 ('mulr', 1, 1, 1)
# registers: [1, 4, 18, 0, 0, 0]
# Running 19 ('mulr', 2, 1, 1)
# registers: [1, 76, 19, 0, 0, 0]
# Running 20 ('muli', 1, 11, 1)
# registers: [1, 836, 20, 0, 0, 0]
# Running 21 ('addi', 4, 3, 4)
block_B(0)

836

In [5]:
# Block_C
# 21 addi 4 3 4
# 22 mulr 4 2 4
# 23 addi 4 7 4
# 24 addr 1 4 1
# registers: [1, 836, 20, 0, 0, 0]
# Running 21 ('addi', 4, 3, 4)
# registers: [1, 836, 21, 0, 3, 0]
# Running 22 ('mulr', 4, 2, 4)
# registers: [1, 836, 22, 0, 66, 0]
# Running 23 ('addi', 4, 7, 4)
# registers: [1, 836, 23, 0, 73, 0]
# Running 24 ('addr', 1, 4, 1)
# registers: [1, 909, 24, 0, 73, 0]
def block_C(buf_1, buf_4):
    x= 21
    buf_4 += 3
    buf_4 *= (x + 1)
    buf_4 += 7
    buf_1 += buf_4
    return buf_1    

In [6]:
block_C(836, 0)

909

In [7]:
# Block D
# 1  seti 1 1 5    -> reg[5] = 1
# 2  seti 1 1 3    -> reg[3] = 1
# 3  mulr 5 3 4    -> reg[4] = reg[3] * reg[5]
# 4  eqrr 4 1 4    -> reg[4] = reg[4] == reg[1]
# 5  addr 4 2 2  * -> Would be stuck here if
# 6  addi 2 1 2  * -> 8
# 7  addr 5 0 0    # THis is the one that break out?
# 8  addi 3 1 3
# 9  gtrr 3 1 4
# 10 addr 2 4 2  *
# 11 seti 2 8 2  * -> 3

# registers: [0, 10551309, 0, 0, 10550400, 0]
# Running 1 ('seti', 1, 1, 5)
# registers: [0, 10551309, 1, 0, 10550400, 1]
# Running 2 ('seti', 1, 1, 3)
# registers: [0, 10551309, 2, 1, 10550400, 1]
# Running 3 ('mulr', 5, 3, 4)
# registers: [0, 10551309, 3, 1, 1, 1]
# Running 4 ('eqrr', 4, 1, 4)
# registers: [0, 10551309, 4, 1, 0, 1]
# Running 5 ('addr', 4, 2, 2)               -> 6 or 7
# registers: [0, 10551309, 5, 1, 0, 1]
# Running 6 ('addi', 2, 1, 2)
# registers: [0, 10551309, 7, 1, 0, 1]
# Running 8 ('addi', 3, 1, 3)
# registers: [0, 10551309, 8, 2, 0, 1]
# Running 9 ('gtrr', 3, 1, 4)
# registers: [0, 10551309, 9, 2, 0, 1]
# Running 10 ('addr', 2, 4, 2)
# registers: [0, 10551309, 10, 2, 0, 1]
# Running 11 ('seti', 2, 8, 2)
# registers: [0, 10551309, 2, 2, 0, 1]
# Running 3 ('mulr', 5, 3, 4)
# registers: [0, 10551309, 3, 2, 2, 1]

# 12 addi 5 1 5
# 13 gtrr 5 1 4
# 14 addr 4 2 2  *
# 15 seti 1 5 2  * -> 2
# 16 mulr 2 2 2  *
def block_D(reg_0, reg_1):
    x = 1

    reg_5 = 1                      #1
    
    while True:                    #15
        reg_3 = 1                  #2

        while True:                #11
            reg_4 = reg_3 * reg_5  #3

            if reg_4 == reg_1:     #4, 5 and 6
                reg_0 += reg_5     #7

            reg_3 += 1             #8

            if reg_3 > reg_1:      # 9  goto 12 else 11
                break              # 10
        
        reg_5 += 1                 # 12
        
        if reg_5 > reg_1:          # 13
            break                  # 14

    return reg_0

    

Block D increments reg_0 by each of the divisors of reg_1!

In [8]:
import math
def divisors(n):
    divs = [1]
    for i in range(2,int(math.sqrt(n))+1):
        if n%i == 0:
            divs.extend([i,n/i])
    divs.extend([n])
    return set(divs)

In [10]:
def block_D_improved(reg_1):
    return int(sum(divisors(reg_1)))

In [11]:
# Check
for i in range(1, 31):
    print(i, block_D(0, i), block_D_improved(i))

1 1 1
2 3 3
3 4 4
4 7 7
5 6 6
6 12 12
7 8 8
8 15 15
9 13 13
10 18 18
11 12 12
12 28 28
13 14 14
14 24 24
15 24 24
16 31 31
17 18 18
18 39 39
19 20 20
20 42 42
21 32 32
22 36 36
23 24 24
24 60 60
25 31 31
26 42 42
27 40 40
28 56 56
29 30 30
30 72 72


In [12]:
block_D_improved(10551309)

14562240

In [46]:
# Putting it all together
def full_program(reg_0=0, reg_1=0, reg_4=0, part_1=False):
    reg_1 = block_B(reg_1)
    print('reg_1', reg_1)
    reg_1 = block_C(reg_1, reg_4)
    print('reg_1', reg_1)
    
    if not part_1:
        # Block A is skipped in part 1
        reg_1 += 10550400
        print('reg_1', reg_1)

    # Block D
    reg_0 += block_D_improved(reg_1)
    
    return reg_0


In [47]:
full_program()

reg_1 836
reg_1 909
reg_1 10551309


14562240